<a href="https://colab.research.google.com/github/dagobertomedeiros/curso_analise_series_temporais/blob/main/curso_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd